Gather all the analysed subjects (quality)

In [ ]:
import pandas as pd
import numpy as np
import os

### N=83 cohort

In [ ]:
ratings_df1 = pd.read_excel('/home/jaimebarranco/Desktop/MRI-QC/scores.xlsx', sheet_name='brainmask_avg_data')

In [ ]:

# add another column which is the opposite of 'exclusion' column
ratings_df1['rate'] = ratings_df1['exclusion'].apply(lambda x: 1 if x == 0 else 0)

# remove all the columns except for 'subject' and 'rate'
ratings_df1 = ratings_df1[['subject', 'rate', 'rating_text']]

# sort by 'name'
ratings_df1 = ratings_df1.sort_values(by=['subject'])
ratings_df1['subject'] = ratings_df1['subject'].apply(lambda x: x.split('.')[0])

Get bids name

In [ ]:
'''
Get bids name
'''
subs_ls = '/mnt/sda1/Repos/a-eye/Data/SHIP_dataset/non_labeled_dataset'
files_1 = [(os.path.basename(f)).split('.')[0] for f in sorted(os.listdir(subs_ls)) if not f.startswith('.')]
files_1 = [int(f) for f in files_1] # files from string to int
numbers_1 = [f for f in range(1, len(files_1)+1)]
df_1 = pd.DataFrame({'Subject': files_1, 'Number': numbers_1})

subs = '/home/jaimebarranco/Downloads/samples_v3'
files_2 = [(os.path.basename(f)).split('.')[0] for f in sorted(os.listdir(subs)) if not f.startswith('.')]
files_2 = [int(f) for f in files_2] # files from string to int
numbers_2 = [f for f in range(1, len(files_2)+1)]
df_2 = pd.DataFrame({'Subject': files_2, 'Number': numbers_2})

# df_1 only with df_2 subjects
df_3 = df_1[df_1['Subject'].isin(df_2['Subject'])]
numbers_3 = df_3['Number']
bids_names = [f'sub-{n:03}_T1w' for n in numbers_3]
df_3['bids_name'] = bids_names # add bids names
df_3 = df_3.reset_index(drop=True) # reset index
df_3 = df_3.sort_values(by=['Subject']) # sort by 'Subject'

In [ ]:
# add 'bids_name' column from df_3 to ratings_df1
ratings_df1['bids_name'] = df_3['bids_name'].values
ratings_df1 = ratings_df1.sort_values(by=['bids_name'])
ratings_df1 = ratings_df1.reset_index(drop=True) # reset index

In [ ]:
# move columns
ratings_df1.insert(0, 'bids_name', ratings_df1.pop('bids_name'))
ratings_df1 = ratings_df1[['bids_name', 'rate', 'rating_text']]
ratings_df1 = ratings_df1.rename(columns={'rating_text': 'comments'})
ratings_df1['bids_name'] = ratings_df1['bids_name'].apply(lambda x: x.split('_')[0])

Get reports name from LS dataset

In [ ]:
bids_csv = pd.read_csv("/home/jaimebarranco/Desktop/MRI-QC/fetal/fetalqc_non-labeled-dataset/bids_csv.csv")
bids_csv['im'] = bids_csv['im'].apply(lambda x: x.split('/')[-1].split('_')[0])
# rename 'im' column to 'bids_name'
bids_csv = bids_csv.rename(columns={'im': 'bids_name'})

In [ ]:
# get the subset of bids_csv where 'name' is present
bids_csv_name = bids_csv[bids_csv['name'].notnull()]
# merge ratings_df1 and bids_csv_name on 'bids_name'
merged_df1 = pd.merge(ratings_df1, bids_csv_name, on='bids_name')
# add the 'name' column from bids_csv_name to ratings_df1
ratings_df1['name'] = merged_df1['name']
# reorder columns
ratings_df1.insert(1, 'name', ratings_df1.pop('name'))
# order by 'name'
ratings_df1 = ratings_df1.sort_values(by=['name'])

In [ ]:
ratings_df1 = ratings_df1.rename(columns={'name': 'report_name'})
ratings_df1 = ratings_df1.reset_index(drop=True) # reset index

### N=100 cohort

excluded_edited

In [ ]:
ratings_df2 = pd.read_excel('/home/jaimebarranco/Desktop/MRI-QC/excluded_edited.xlsx')

In [ ]:
# remove rows where 'my_rate' is nan
ratings_df2 = ratings_df2[ratings_df2['my_rate'].notnull()]
# keeping only the columns we want
ratings_df2 = ratings_df2[['subject_name', 'bids_name', 'my_rate', 'Comments']]
# reorder columns
ratings_df2.insert(0, 'bids_name', ratings_df2.pop('bids_name'))
# renaming columns
ratings_df2 = ratings_df2.rename(columns={'subject_name': 'report_name', 'my_rate': 'rate', 'Comments': 'comments'})
# changing rate according to mriqc-learn
ratings_df2['rate'] = ratings_df2['rate'].apply(lambda x: 1 if x == 0 else 0)
# changing bids_name
ratings_df2['bids_name'] = ratings_df2['bids_name'].apply(lambda x: x.split('_')[0])
# order by 'name'
ratings_df2 = ratings_df2.sort_values(by=['report_name'])
ratings_df2 = ratings_df2.reset_index(drop=True) # reset index

common subs between dataframes

In [ ]:
# count how many elements are in common between ratings_df1 and ratings_df2 by 'report_name'
common_df1_df2 = ratings_df1[ratings_df1['report_name'].isin(ratings_df2['report_name'])]
common_df1_df2 = common_df1_df2.sort_values(by=['report_name'])
common_df1_df2 = common_df1_df2.reset_index(drop=True) # reset index
print(common_df1_df2)

common_df2_df1 = ratings_df2[ratings_df2['report_name'].isin(ratings_df1['report_name'])]
common_df2_df1 = common_df2_df1.sort_values(by=['report_name'])
common_df2_df1 = common_df2_df1.reset_index(drop=True) # reset index
print(common_df2_df1)

In [ ]:
for i in range(len(common_df1_df2)):
    if common_df1_df2['rate'].values[i] != common_df2_df1['rate'].values[i]:
        print(f'{common_df1_df2["report_name"].values[i]} has {common_df1_df2["rate"].values[i]} in excel1 and {common_df2_df1["rate"].values[i]} in excel2')

merge dataframes

In [ ]:
# concat ratings_df1 and ratings_df2. If 'rate' is different in both, the value from ratings_df1 will be used
ratings_df = pd.concat([ratings_df1, ratings_df2])
ratings_df = ratings_df.sort_values(by=['report_name'])
ratings_df = ratings_df.reset_index(drop=True) # reset index
ratings_df = ratings_df.drop_duplicates(subset=['report_name'], keep='first') # drop duplicates

### All other excel files

#### Excels

In [ ]:
excels = [
    '/home/jaimebarranco/Downloads/excluded_mriqclearn.xlsx',
    '/home/jaimebarranco/Downloads/excluded_mriqclearn_N183_NoBrainIQMs_th043.xlsx',
    '/home/jaimebarranco/Downloads/excluded_mriqclearn_N183_NoBrainIQMs_th0389.xlsx',
    '/home/jaimebarranco/Downloads/excluded_mriqclearn_NoBrainIQMs_th0336.xlsx',
    '/home/jaimebarranco/Downloads/excluded_mriqclearn_trN183_th0753.xlsx',
    '/home/jaimebarranco/Downloads/excluded_N183_NoBrainIQMs.xlsx',
]

#### Functions

Preprocess

In [ ]:
def preprocess_df(df):
    '''
    Preprocess the dataframe
    '''
    # remove rows where 'my_rate' is nan
    df = df[df['my_rate'].notnull()]
    # renaming columns
    df = df.rename(columns={'name': 'report_name', 'my_rate': 'rate'})
    # changing bids_name
    df['bids_name'] = df['bids_name'].apply(lambda x: x.split('_')[0])
    # order by 'name'
    df = df.sort_values(by=['report_name'])
    df = df.reset_index(drop=True) # reset index

    return df

Merge dataframes altogether excluding subjects with different rates

In [ ]:
def merge_dataframes(df_list):
    '''
    Merge a list of dataframes keeping the rate from the first dataframe in case of conflict
    '''
    # concat all dataframes in the list
    df_aux = pd.concat(df_list)
    df_aux = df_aux.sort_values(by=['report_name'])
    df_aux = df_aux.reset_index(drop=True) # reset index
    duplicates = df_aux[df_aux.duplicated(subset=['report_name'], keep=False)]
    df_aux = df_aux.drop_duplicates(subset=['report_name'], keep='first') # drop duplicates choosing the first one
    df = df_aux

    # keep track of duplicates with different rates
    common_subs = duplicates[duplicates.duplicated(subset=['report_name'], keep=False)]
    common_subs = common_subs.sort_values(by=['report_name'])
    common_subs = common_subs.reset_index(drop=True)

    # filter diff_rates to only include duplicates where rate differs
    diff_rates = common_subs[~common_subs.duplicated(subset=['report_name', 'rate'], keep=False)]
    diff_rates = diff_rates.drop_duplicates(subset=['report_name'], keep='first')
    diff_rates = diff_rates.drop(columns=['rate', 'comments'])
    diff_rates = diff_rates.sort_values(by=['report_name'])
    diff_rates = diff_rates.reset_index(drop=True)

    return df, common_subs, diff_rates

Get subjects with different rates between dataframes

In [ ]:
def merge_common_diff(df1, df2):
    '''
    Merge common_subs and diff_rates dataframes on 'report_name' column
    '''
    merged_df = pd.merge(df2, df1[['report_name']], on='report_name', how='inner')
    print(merged_df)

    return merged_df


#### Pipeline

Read and preprocess excel files

In [ ]:
ratings_df_aux = [ratings_df]
for ex in range(len(excels)):
    globals()['ratings_df%s' % str(ex+3)] = pd.read_excel(excels[ex])
    globals()['ratings_df%s' % str(ex+3)] = preprocess_df(globals()['ratings_df%s' % str(ex+3)])
    ratings_df_aux.append(globals()['ratings_df%s' % str(ex+3)])

Get common subjects and rates that differ

In [ ]:
# common_df_dfaux, common_dfaux_df = get_common_subs(ratings_df, ratings_df_aux)
# ratings_df, common_subs, diff_rates = merge_dataframes(ratings_df, ratings_df3, ratings_df4, ratings_df5, ratings_df6, ratings_df7)
ratings_df, common_subs, diff_rates = merge_dataframes(ratings_df_aux)

Copy reports to an output folder

In [ ]:
# import os, shutil

# reports_folder = '/home/jaimebarranco/Desktop/MRI-QC/fetal/fetalqc_non-labeled-dataset'
# output_folder = '/home/jaimebarranco/Downloads/all_excluded'
# if not os.path.exists(output_folder):
#     os.makedirs(output_folder)

# # copy html reports from reports_folder that match the subjects in excluded dataframe to output_folder
# for i in range(len(ratings_df)):
#     subject = ratings_df['report_name'].values[i]
#     for filename in os.listdir(reports_folder):
#         if filename.startswith(f'{subject}_report'):
#             shutil.copy(f'{reports_folder}/{filename}', f'{output_folder}/{filename}')

Get subjects with different rates

In [ ]:
subs_diff_rate = merge_common_diff(diff_rates, common_subs) # subdataframe into dataframe

In [ ]:
#  Group the dataframe by report_name
grouped = subs_diff_rate.groupby('report_name')

for name, group in grouped:
    print(name)
    print(group)

    # percent agreement for each group
    percent_agreement = group['rate'].value_counts(normalize=True) * 100
    print(f'percent agreement: \n{percent_agreement}')
    
    print('\n')

In [ ]:
# count how many '0' rates are in ratings_df
print(f"Number of '0' rates: {len(ratings_df[ratings_df['rate'] == 0])}/{len(ratings_df)}")

In [ ]:
# ratings_df removing rows where 'report_name' is in 'diff_rates'
ratings_df_without_diff_rates = ratings_df[~ratings_df['report_name'].isin(diff_rates['report_name'])]

In [ ]:
# count how many '0' rates are in ratings_df
print(f"Number of '0' rates: {len(ratings_df_without_diff_rates[ratings_df_without_diff_rates['rate'] == 0])}/{len(ratings_df_without_diff_rates)}")

# print those subjects ordered by bids_name
print(ratings_df_without_diff_rates[ratings_df_without_diff_rates['rate'] == 0])